<a href="https://colab.research.google.com/github/ericodle/GINSA/blob/main/RUN_GINSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required packages

In [1]:
!pip install requests
!pip install wget
!pip install biopython

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=6fd248b5882a92450a7558c2c28ba7917dfcafb95516d794b14d2a73f453f711
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.5 MB/s eta 0:00:00


# Import dependent packages

In [ ]:
import urllib.request
import json
import urllib.parse
import os
import csv
import requests
import pandas as pd
import wget
import gzip
import shutil
from Bio import SeqIO

# Script functions

In [19]:
###############################################################################################################################

def search_species_occurrences(species_name, limit=300):
    base_url = "https://api.gbif.org/v1/occurrence/search"
    params = {
        "scientificName": species_name,
        "limit": limit,
    }

    occurrence_ids = []

    offset = 0
    while True:
        params["offset"] = offset
        encoded_params = urllib.parse.urlencode(params)
        full_url = f"{base_url}?{encoded_params}"

        with urllib.request.urlopen(full_url) as response:
            data = json.loads(response.read())

            if "results" in data:
                occurrence_ids.extend([occurrence["key"] for occurrence in data["results"]])
                if len(data["results"]) < limit:
                    break  # Reached the end of results
                else:
                    offset += limit
            else:
                break

    return occurrence_ids

###############################################################################################################################

def create_folders(occurrence_ids, proj_dir):
    if not os.path.exists(proj_dir):
        os.makedirs(proj_dir)

    for occurrence_id in occurrence_ids:
        folder_path = os.path.join(proj_dir, str(occurrence_id))
        os.makedirs(folder_path, exist_ok=True)

###############################################################################################################################

def generate_csv(occurrence_ids, proj_dir):
    with open(proj_dir+'/occurrences.csv', mode='w', newline='') as csv_file:
        fieldnames = ['occurrence_id', 'latitude', 'longitude', 'prefix_text']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        writer.writeheader()

        for occurrence_id in occurrence_ids:
            url = f"https://api.gbif.org/v1/occurrence/{occurrence_id}"
            response = requests.get(url)

            if response.status_code == 200:
                try:
                    data = response.json()
                    long_id = data.get("occurrenceID")
                    latitude = data.get("decimalLatitude")
                    longitude = data.get("decimalLongitude")
                    underscore_index = long_id.find("_")
                    if underscore_index != -1:
                        prefix_text = long_id[:underscore_index]
                    else:
                        prefix_text = "N/A"
                    writer.writerow({
                        'occurrence_id': occurrence_id,
                        'latitude': latitude,
                        'longitude': longitude,
                        'prefix_text': prefix_text
                    })
                except ValueError:
                    print(f"Failed to parse JSON for occurrence ID {occurrence_id}.")
            else:
                print(f"Failed to fetch content for occurrence ID {occurrence_id}. Status code: {response.status_code}")

    print("CSV file created successfully.")

###############################################################################################################################

def process_directory(proj_dir):
    for root, dirs, files in os.walk(proj_dir):
        for file in files:
            if file.endswith(".gz"):
                gz_file_path = os.path.join(root, file)
                extract_path = os.path.splitext(gz_file_path)[0]

                with gzip.open(gz_file_path, 'rb') as f_in, open(extract_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)

                os.remove(gz_file_path)
                print(f"Extracted and deleted: {gz_file_path}")

###############################################################################################################################

def ssu_fasta_grab(csv_file, proj_dir):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Replace with your actual MGnify API endpoint
    api_base_url = "https://www.ebi.ac.uk/metagenomics/api/v1"

    # Iterate through the DataFrame and process each occurrence
    for prefix_text, occurrence_id in zip(df['prefix_text'], df['occurrence_id']):
        url = f"{api_base_url}/analyses/{prefix_text}/downloads"
        response = requests.get(url)
        data = response.json()

        print(f"Processing {prefix_text}...")

        # Create a directory with the occurrence_id in the project directory
        subdir_path = os.path.join(proj_dir, str(occurrence_id))
        if not os.path.exists(subdir_path):
            os.makedirs(subdir_path)

        fasta_links = []

        # Loop through the data to find links ending with "MERGED_FASTQ_SSU.fasta.gz"
        for entry in data['data']:
            link_entry = entry['links']['self']
            if link_entry.endswith("MERGED_FASTQ_SSU.fasta.gz"):
                fasta_links.append(link_entry)

        if fasta_links:
            print(f"Downloading {len(fasta_links)} file(s) for {prefix_text}...")
            for fasta_link in fasta_links:
                file_name = os.path.basename(fasta_link)
                file_path = os.path.join(subdir_path, file_name)
                wget.download(fasta_link, file_path)
            print("Download complete.")
        else:
            print(f"No fasta files found for {prefix_text}. Saving 'no_fasta.txt'...")
            no_fasta_file = os.path.join(subdir_path, "no_fasta.txt")
            with open(no_fasta_file, "w") as f:
                f.write("No fasta files found.")

        print()  # Add an empty line for better separation

###############################################################################################################################

def mapseq_grab(csv_file, proj_dir):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Replace with your actual MGnify API endpoint
    api_base_url = "https://www.ebi.ac.uk/metagenomics/api/v1"

    # Iterate through the DataFrame and process each occurrence
    for prefix_text, occurrence_id in zip(df['prefix_text'], df['occurrence_id']):
        url = f"{api_base_url}/analyses/{prefix_text}/downloads"
        response = requests.get(url)
        data = response.json()

        print(f"Processing {prefix_text}...")

        # Create a directory with the occurrence_id in the project directory
        subdir_path = os.path.join(proj_dir, str(occurrence_id))
        if not os.path.exists(subdir_path):
            os.makedirs(subdir_path)

        mapseq_links = []

        # Loop through the data to find links ending with "MERGED_FASTQ_SSU_MAPSeq.mseq.gz"
        for entry in data['data']:
            link_entry = entry['links']['self']
            if link_entry.endswith("MERGED_FASTQ_SSU_MAPSeq.mseq.gz"):
                mapseq_links.append(link_entry)

        if mapseq_links:
            print(f"Downloading {len(mapseq_links)} file(s) for {occurrence_id}...")
            for mapseq_link in mapseq_links:
                file_name = os.path.basename(mapseq_link)
                file_path = os.path.join(subdir_path, file_name)
                wget.download(mapseq_link, file_path)
            print("Download complete.")
        else:
            print(f"No MAPSeq files found for {occurrence_id}. Saving 'no_mapseq.txt'...")
            no_mapseq_file = os.path.join(subdir_path, "no_mapseq.txt")
            with open(no_mapseq_file, "w") as f:
                f.write("No MAPSeq files found.")

        print()  # Add an empty line for better separation

###############################################################################################################################

def count_occurrences_in_mapseq_file(file_path, species):

    try:
        with open(file_path, 'r') as file:
            content = file.read()

            # Count the occurrences of the species name in the entire file
            num_occurrences = content.count(species)

            if num_occurrences == 0:
                print(f"'{species}' not found in the MAPSEQ file.")
            else:
                print(f"Number of occurrences of '{species}': {num_occurrences}")

            return num_occurrences

    except FileNotFoundError:
        print("Error: The file does not exist or the path is incorrect.")
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0

###############################################################################################################################

def find_target_in_mapseq(subdirectory, file_path, species):

    truncated_strings = []  # List to store truncated strings

    try:
        with open(file_path, 'r') as file:
            content = file.readlines()

            for line in content:
                line = line.strip()
                if species in line:
                    # Find the index of the search word in the line
                    word_index = line.index(species)

                    # Extract the preceding string
                    preceding_string = line[:word_index].strip()
                    # Truncate to only the text before the first space
                    truncated_string = preceding_string.split()[0]
                    truncated_strings.append(truncated_string)

        with open(subdirectory+"/truncated_strings.txt", 'w') as output_file:
            for item in truncated_strings:
                output_file.write("%s\n" % item)

    except FileNotFoundError:
        print("MAPSeq Search Error: The file does not exist or the path is incorrect.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return truncated_strings

###############################################################################################################################

def sift_fasta(fasta_file_path, truncated_strings):
    sifted_data = {}  # Dictionary to store truncated_strings and their corresponding DNA sequences

# Parse the FASTA file and extract sequences
    with open(fasta_file_path, "rt") as fasta_file:
        records = SeqIO.parse(fasta_file, "fasta")
        for record in records:
            sequence_name = record.id
            sequence = str(record.seq)

# Search for sequence names in the truncated_strings list
        for truncated_string in truncated_strings:
            if truncated_string in sequence_name:
                sifted_data.setdefault(truncated_string, []).append(sequence)
                break

    return sifted_data
###############################################################################################################################

def save_sifted_data_to_csv(sifted_data, output_file):
    with open(output_file, "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Truncated String", "Sequences"])

        for truncated_string, sequences in sifted_data.items():
            csv_writer.writerow([truncated_string, "\n".join(sequences)])

###############################################################################################################################

def analyze_subdir_mapseq(proj_dir, species):
    # Iterate through subdirectories in proj_dir
    for subdirectory in os.listdir(proj_dir):
        subdir_path = os.path.join(proj_dir, subdirectory)

        # Skip if not a directory
        if not os.path.isdir(subdir_path):
            continue

        print(f"Analyzing subdirectory: {subdirectory}")

        # Check for required files and txt file
        for file_name in os.listdir(subdir_path):
            if file_name.endswith("MERGED_FASTQ_SSU_MAPSeq.mseq"):

                print(file_name)
              # Apply the count_occurrences_in_mapseq_file function
                count_occurrences_in_mapseq_file(subdir_path+"/"+file_name, species)
                find_target_in_mapseq(subdir_path, subdir_path+"/"+file_name, species)
                print("Truncated strings obtained from MAPSeq file.")
            else:
                print("No MAPSeq files found.")

###############################################################################################################################


def analyze_subdir_fasta(proj_dir):
    # Iterate through subdirectories in proj_dir
    for subdirectory in os.listdir(proj_dir):
        subdir_path = os.path.join(proj_dir, subdirectory)

        print("subdirectory path:", subdir_path)

        # Skip if not a directory
        if not os.path.isdir(subdir_path):
            print(f"Skipping non-directory: {subdirectory}")
            continue

        print(f"Analyzing subdirectory: {subdirectory}")

        fasta_files = [file_name for file_name in os.listdir(subdir_path) if file_name.endswith("MERGED_FASTQ_SSU.fasta")]

        # Check if any FASTA files are found in the subdirectory
        if fasta_files:
            # Iterate through FASTA files in the subdirectory
            for fasta_file in fasta_files:
                fasta_filepath = os.path.join(subdir_path, fasta_file)
                print("Found FASTA file:", fasta_filepath)

                # Construct the path for the truncated strings file
                truncated_strings_filepath = os.path.join(subdir_path, "truncated_strings.txt")

                # Perform analysis using sift_fasta and save_sifted_data_to_csv functions
                sifted_data = sift_fasta(fasta_filepath, truncated_strings_filepath)

                # Create a new CSV filename based on the original FASTA filename
                csv_filename = os.path.splitext(fasta_file)[0] + "_extracted_sequences.csv"

                # Save the sifted data to the new CSV file
                save_sifted_data_to_csv(sifted_data, os.path.join(subdir_path, csv_filename))

            print("Extraction and CSV creation complete for all FASTA files.")
        else:
            print("No FASTA files found in the subdirectory.")

###############################################################################################################################

# Main control function

In [20]:
def main():

    print("Welcome to the GbIf Next-gen Sequence Analyzer! (GINSA)")

    proj_dir = input("Enter the path to an empty folder for this project.")

    species_name = input("Enter your target taxon in genus-species format (i.e. Lecudina tuzetae): ")

    gen_sp = species_name.split()

    genus = gen_sp[0]
    print("Genus: ", genus)

    species = gen_sp[1]
    print("Species: ", species)

    occurrence_ids = search_species_occurrences(species_name)

    num_occurrences = len(occurrence_ids)

    if num_occurrences > 0:
        print(f"Number of occurrences found: {num_occurrences}")

    else:
        print("No occurrences found for the species.")

    create_folders(occurrence_ids, proj_dir)
    print("Subdirectory folders created")

    generate_csv(occurrence_ids, proj_dir)
    print("Occurrence metadata CSV created")

    csv_file = proj_dir+"/occurrences.csv"

    print("Grabbing FASTA and MAPSeq files from GBIF and MGnify")
    ssu_fasta_grab(csv_file, proj_dir)

    mapseq_grab(csv_file, proj_dir)


    print("Unpacking compressed files")
    process_directory(proj_dir)

    analyze_subdir_mapseq(proj_dir, species)

    analyze_subdir_fasta(proj_dir)

    print("Analysis complete! Sifted sequences can be found in their respective sub-folders")

# Run the script!

In [ ]:
if __name__ == "__main__":
    main()